In [6]:
# cpu
# 드라이브 연동
# cd 딥러닝 폴더 경로 이동
# 기본경로 설정?
%cd /content/drive/MyDrive/Colab Notebooks/DeepLearning

/content/drive/MyDrive/Colab Notebooks/DeepLearning


### 목표
  - 개, 고양이 이미지(사진)을 배열로 변환하는 작업을 해보자(이미지 데이터 전처리)
    - 압축된 형식의 배열로 저장: npz
    -  대용량의 데이터를 압축형태로 정리해두면 클라우드 환경에 쉽게 업로드, 다운로드가 가능해짐(데이터 관리 편해짐)
    - 저장소의 공간을 적게 차지함(공간 사용 효율이 좋아짐)

In [ ]:
# 1. 압축 폴더 해제
# 2. 이미지 한장씩 접근
# 3. 이미지 크기 조정(동일하게)
# 4. 배열 형태 변환(img->nparray)
# 5. 모든 배열 데이터에 대해서 npz로 정리

In [2]:
# PIL 에러
!pip install pillow

In [4]:
# 환경셋팅
from zipfile import ZipFile
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image # 설치가 필요할 수 있음

In [7]:
# 1. 압축 폴더 해제 - ZipFile
zip_data = './data/cats_and_dogs_filtered.zip'

# 파일 다루는 구문
# with open('경로설정,'모드') as f:
with ZipFile(zip_data, 'r') as f:
    f.extractall(path='./data/')

In [13]:
# 경로 정리
# train - cats, dogs (2개)
# test - cats, dogs(2개)
# 총 4개 폴더 경로 정리
train_cats_dir = './data/cats_and_dogs_filtered/train/cats'
train_dogs_dir = './data/cats_and_dogs_filtered/train/dogs'

test_cats_dir = './data/cats_and_dogs_filtered/test/cats'
test_dogs_dir = './data/cats_and_dogs_filtered/test/dogs'

In [14]:
# 각 폴더 안에 있는 이미지 파일이름 확인해서 저장하기
# os.listdir(폴더경로): 해당경로에 있는 파일명을 리스트로 출력
train_cats_fn = os.listdir(train_cats_dir)
train_dogs_fn = os.listdir(train_dogs_dir)
test_cats_fn = os.listdir(test_cats_dir)
test_dogs_fn = os.listdir(test_dogs_dir)

In [15]:
# 리스트 길이 확인
print(len(train_cats_fn), len(train_dogs_fn)) # 2000장 이미지
print(len(test_cats_fn), len(test_dogs_fn)) # 1000장 이미지

1000 1000
500 500


In [24]:
# 2. 이미지 한장씩 접근
# 예시) 1장 접근해서 이미지 출력해보기 (train > cats > 0번째 이미지)
# 폴더 경로 + 이미지 이름 경로 : os.path.join()
tmp_path = os.path.join(train_cats_dir, train_cats_fn[2])
tmp_path

# 3. 이미지 크기 조정(동일하게)
img = Image.open(tmp_path).resize((224,224)) # resieze((행,열))
type(img)

# 4. 배열 형태 변환(img->nparray)
img_arr = np.array(img)
type(img_arr)

numpy.ndarray

#### 이미지 처리하는 함수 정의
  - 2~4번 연결

In [29]:
def prepro_imgs(folder_path, file_names, img_size=(224,224)):
  # 최종적으로 이미지를 담아서 리턴할 빈 리스트
  img_lst = [] # 1장씩 배열로 변환하고 누적할 공간

  for fn in file_names:
    # fn 이미지 한장 이름
    # 1. 폴더명 + 파일명 연결 /안써도 알아서 경로 연결해줌
    img_path = os.path.join(folder_path, fn)
    # 2. 이미지 형태로 불러오기 + 크기 조정
    img = Image.open(tmp_path).resize(img_size)

    # 3. 배열 변환 (img>nparray)
    img_arr = np.array(img)

    # img_lst 누적 (뒤로 추가: list.append(대상))
    img_lst.append(img_arr)

  return np.array(img_lst)


In [30]:
# 함수 정의후 호출해서 사용하기
# prepro_imgs(train_cats_dir, train_cats_fn) 호출해서 변수에 담고 크기 확인하기
X_train_cats = prepro_imgs(train_cats_dir, train_cats_fn)
X_train_dogs = prepro_imgs(train_dogs_dir, train_dogs_fn)
X_test_cats = prepro_imgs(test_cats_dir, test_cats_fn)
X_test_dogs = prepro_imgs(test_dogs_dir, test_dogs_fn)

# 크기확인
print('훈련데이터:', X_train_cats.shape, X_train_dogs.shape)
print('테스트데이터:', X_test_cats.shape, X_test_dogs.shape)

# (이미지개수, 행, 열, 채널크기)
# 채널크기 3: 컬러이미지
# 채널크기 1: 흑백이미지

훈련데이터: (1000, 224, 224, 3) (1000, 224, 224, 3)
테스트데이터: (500, 224, 224, 3) (500, 224, 224, 3)


In [32]:
# 문제 데이터 병합
# train 기준으로 2개 데이터 병합
# c + d
X_train = np.concatenate((X_train_cats, X_train_dogs))
# test
X_test = np.concatenate((X_test_cats, X_test_dogs))

# 크기확인
print('훈련데이터:', X_train.shape)
print('테스트데이터:', X_test.shape)

훈련데이터: (2000, 224, 224, 3)
테스트데이터: (1000, 224, 224, 3)


In [34]:
# cats - 0, dogs - 1
# 답 데이터 배열로 생성
y_train = np.array([0]*1000 + [1]*1000)
print(y_train)
# y_test 생성해보기
y_test = np.array([0]*500 + [1]*500)
print('훈련답:', y_train.shape)
print('테스트답: ', y_test.shape)

[0 0 0 ... 1 1 1]
훈련답: (2000,)
테스트답:  (1000,)


In [36]:
# npz 형태로 저장하기
# X_train, X_test
# y_train, y_test
np.savez_compressed('./data/np_cats_dogs.npz',
                    X_train = X_train,
                    X_test = X_test,
                    y_train = y_train,
                    y_test = y_test)